# ⭐⭐⭐模型构造：层与块

**基础回顾非常重要**

---

## 一、神经网络到底在干什么？

```
数据进来 → 经过第1层 → 经过第2层 → 经过第3层 → 结果出来
```


## 二、Python基础再rev

### 类class：一张图纸

```python
# "类"就是一个模板/图纸
class Dog:
    def __init__(self, name):      # 造狗时要做的准备
        self.name = name           # 把名字记住（绑在自己身上）
    
    def bark(self):                # 狗会叫
        print("你好，我是" + self.name)

# 用图纸造一只真狗
my_dog = Dog("旺财")
my_dog.bark()                      # 输出：你好，我是旺财
```

| 概念 | 意思 | 比喻 |
|:---|:---|:---|
| `class` | 定义一个模板 | 画图纸 |
| `__init__` | 造东西时要做的准备初始化 | 买材料 |
| `self` | "我自己" | 指代这只狗自己 |
| `self.name` | 我自己的名字 | 这只狗记住自己叫什么 |
| `Dog("旺财")` | 用模板造一个实际的东西 | 按图纸造一只真狗 |

---

## 三、搞懂 self

### 3.1 self = "我自己的"

```python
self.hidden = nn.Linear(20, 256)
# 意思：我自己的hidden层 = 一个吃20吐256的机器
```

### 3.2 为什么需要self？

```python
# ❌ 没有self
def __init__(self):
    hidden = nn.Linear(20, 256)    # 临时变量，函数结束就没了

def forward(self, X):
    X = hidden(X)                  # 报错！hidden已经消失了！
```

```python
# ✅ 有self
def __init__(self):
    self.hidden = nn.Linear(20, 256)    # 绑在自己身上，一直在

def forward(self, X):
    X = self.hidden(X)                  # 能找到！因为绑在身上
```

**生活比喻：**

```
没有self → 在商店买了菜刀，放在门口就走了 → 回家做菜时找不到
有self   → 买了菜刀带回家了 → 随时能用
```

### 3.3 核心规则

**init里用self存东西，forward里用self取东西，名字必须对应！**

```python
# ✅ 名字对应
self.aaa = nn.Linear(20, 256)     # init里叫aaa
X = self.aaa(X)                    # forward里也叫aaa

# ❌ 名字不对应
self.aaa = nn.Linear(20, 256)     # init里叫aaa
X = self.bbb(X)                    # 报错！没有bbb这个东西！
```

---

## 四、搞懂 nn.Linear

### 4.1 它是一台"加工机器"

```python
machine = nn.Linear(20, 256)
```

```
我造了一台机器
    吃 20 个数字（输入维度）
    吐 256 个数字（输出维度）
我给它起名叫 machine
```

**机器被造出来时，PyTorch自动在里面放好了随机的权重，你不需要手动设置。**

### 4.2 数字必须前后对应

**前一层的输出 = 后一层的输入，就像水管接水管：**

```
你的数据
[20个数] ← 输入

    ↓  nn.Linear(20, 256)      20进，256出

[256个数]

    ↓  nn.Linear(256, 10)      256进，10出
                                这个256必须=上面的256！

[10个数] ← 最终输出
```

```
如果不匹配：
machine1 吐出 256 个数
machine2 只能吃 100 个数
→ 报错！塞不进去！就像粗水管接细水管，接不上！
```

---

## 五、搞懂 F

### 5.1 F是什么？

```python
from torch.nn import functional as F
```

**F就是一个装满现成函数的工具包，起名叫F只是为了少打字。**

```
F.relu()       → ReLU激活函数（负数变0）
F.sigmoid()    → Sigmoid函数
F.softmax()    → Softmax函数
```

### 5.2 F.relu vs nn.ReLU

```
F.relu(X)     → 直接说"帮我切菜"        → 一个动作
nn.ReLU()     → 先买把菜刀放厨房再切     → 一个东西
```

**结果完全一样！ReLU没有要学习的参数，所以用F.relu更省事。**

**简单规则：**

| 情况 | 用什么 | 例子 |
|:---|:---|:---|
| 有参数要学习的层 | nn.XXX | nn.Linear |
| 没参数的纯计算 | F.xxx | F.relu |

---

## 六、哪些能改，哪些不能改

```python
class MLP(nn.Module):
#     ^^^ 你起的名字 ✅能改     
#          nn.Module  ❌不能改（PyTorch规定）

    def __init__(self):          # __init__ ❌不能改（Python规定）
        super().__init__()       # ❌不能改（固定写法）
        self.hidden = nn.Linear(20, 256)
#            ^^^^^^ 你起的名字 ✅能改
#                    nn.Linear ❌不能改（PyTorch提供的层）
#                              ^^ ^^^ 数字 ✅根据需要改
    
    def forward(self, X):        # forward ❌不能改（PyTorch规定）
#                     ^ 你起的名字 ✅能改
        X = self.hidden(X)
#            ^^^^^^ 必须和init里的名字对应！
        X = F.relu(X)            # F.relu ❌不能改（PyTorch提供的函数）
```

---

## 七、两种构造方式

### 7.1 方式一：Sequential（简单版）

**像坐地铁：只能按站走，不能拐弯。**

```python
net = nn.Sequential(
    nn.Linear(20, 256),    # 第1站
    nn.ReLU(),             # 第2站
    nn.Linear(256, 10)     # 第3站
)

X = torch.rand(2, 20)     # 造2个样本，每个20个特征
output = net(X)            # 丢进去，自动按顺序走
```

### 7.2 方式二：自定义类（灵活版）

**像自己开车：想怎么走就怎么走。**

```python
class MLP(nn.Module):
    def __init__(self):     # 我有哪些零件
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)
    
    def forward(self, X):   # 数据怎么走
        X = self.hidden(X)
        X = F.relu(X)
        X = self.out(X)
        return X

net = MLP()      # 造网络【实例化】
output = net(X)    # 丢数据（自动调用forward）
```

### 7.3 什么时候用哪个？

| 场景 | 用什么 |
|:---|:---|
| 简单的一层接一层 | Sequential，省事 |
| 需要if判断、循环、分支等 | 自定义类 |

---

## 八、nn.Module 是一切的基础

```
nn.Linear      → 是 nn.Module 的子类
nn.ReLU        → 是 nn.Module 的子类  
nn.Sequential  → 是 nn.Module 的子类
你自定义的MLP   → 也是 nn.Module 的子类
```

**都是同一个接口，所以可以像乐高一样随意拼接嵌套：**

```python
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()#必须写！！！
        self.net = nn.Sequential(  # 里面放Sequential
            nn.Linear(20, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU()
        )
        self.linear = nn.Linear(32, 16)  # 再加一个单独的层
    
    def forward(self, X):
        return self.linear(self.net(X))   # 先过net再过linear
        # 一定⭐⭐注意执行顺序前向传播由内向外的顺序！
```

**大盒子里放小盒子，小盒子里还能放更小的盒子。**



---

## 十、完整代码逐行翻译

```python
import torch                            # 导入PyTorch工具箱
from torch import nn                    # 拿出神经网络工具
from torch.nn import functional as F    # 拿出函数工具包，叫F

class MLP(nn.Module):                   # 画图纸，叫MLP

    def __init__(self):                 # 准备工作
        super().__init__()              # PyTorch自己的准备（固定写法）
        self.hidden = nn.Linear(20, 256)  # 绑一台机器：吃20吐256
        self.out = nn.Linear(256, 10)     # 绑一台机器：吃256吐10

    def forward(self, X):               # 数据怎么走
        X = self.hidden(X)              # X过hidden层：20→256
        X = F.relu(X)                   # 负数变0
        X = self.out(X)                 # X过out层：256→10
        return X                        # 吐出结果

net = MLP()                             # 按图纸造网络
X = torch.rand(2, 20)                   # 造数据：2个样本×20特征
output = net(X)                         # 丢进网络，输出2×10
```

---

## 终极总结：三句话记住一切

```
1. __init__ 里：用self绑好所有的层（"我有哪些零件"）
2. forward 里：用self取出层，定义数据怎么走（"数据怎么流动"）
3. 反向传播不用管，PyTorch自动搞定
```